In [1]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, WebSearchPreviewTool, ApproximateLocation

load_dotenv()

foundary_project_endpoint = os.getenv("FOUNDARY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

In [2]:
project_client = AIProjectClient(
    endpoint=foundary_project_endpoint,
    credential=DefaultAzureCredential(),
)

In [8]:
# creating an openai client first
openai_client = project_client.get_openai_client()

### Creating an agent with the websearch tool

In [9]:
agent = project_client.agents.create_version(
    agent_name="web-search-agent",
    definition=PromptAgentDefinition(
        model = model_deployment_name,
        instructions = "You are a helpful assistant that uses web search to answer user queries.",
        tools = [
            WebSearchPreviewTool()
        ]
    )
)

In [10]:
# Creating a conversation to use with the agent

conversation = openai_client.conversations.create()
print(f"Created conversation with id:{conversation.id}")

Created conversation with id:conv_56c2461796cd97d800jXwODowRPYhePyNoLa7rLdhE9Pei72iJ


In [11]:
# Chat with the agent

user_query = "What are the latest advancements in renewable energy"

In [12]:
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body={
        "agent": {
            "name": "web-search-agent",
            "type": "agent_reference"
        }
    },
    input = user_query
)

print(f"Agent Response: {response.output_text}")

Agent Response: Here are some of the latest advancements in renewable energy as of 2025:

1. **Solar Technology**: New materials like perovskite are enhancing solar panel efficiency, making solar energy more accessible and cost-effective. Solar energy production is expanding significantly, contributing to a larger share of global electricity [Renewables 2025 Global Status Report](https://www.ren21.net/renewables-2025-global-status-report-global-overview/) [Top 10 Renewable Energy Trends to Watch in 2025](https://renewablesenergysources.com/top-10-renewable-energy-trends-to-watch-in-2025).

2. **Wind Energy**: There's substantial growth in offshore wind farms, which provides high-capacity energy solutions while minimizing land use. Wind energy systems continue to expand, contributing to the shift towards cleaner power sources [Renewable Energy Trends 2025](https://renewablesenergysources.com/top-10-renewable-energy-trends-to-watch-in-2025) [Ember's Highlights of the Global Energy Transi